In [2]:
# load entities
import json

with open('../data/entities.json', 'r') as fin:
    entities = json.load(fin)

corpus = []
for _id, label in entities.items():
    corpus.append({"_id": _id, "title": "", "text": label, "metadata": {}})

print(len(corpus), 'entities')
print(corpus[0])

28497 entities
{'_id': 'Q1938494', 'title': '', 'text': 'Mirosław Bork', 'metadata': {}}


In [4]:
# load data from https://github.com/askplatypus/wikidata-simplequestions
data_path = '/ivi/ilps/personal/svakule/spoken_qa/'

path_to_train_questions = data_path + 'annotated_wd_data_train.txt'
path_to_valid_questions = data_path + 'annotated_wd_data_valid.txt'

In [7]:
# filter questions with entities for which we have label
dataset = 'WD18_entities/'

corpus_path = data_path + dataset + "corpus.jsonl"
query_path = data_path + dataset + "queries.jsonl"
train_qrels_path = data_path + dataset + "qrels/train.tsv"
valid_qrels_path = data_path + dataset + "qrels/valid.tsv"

queries = []

with open(path_to_train_questions) as fin:
    lines = fin.readlines()
    train_qrels = []
    for i, l in enumerate(lines):
        s, p, o, q = lines[i].strip('\n').split('\t')
        _id = 't' + str(i)
        if s in entities:
            queries.append({"_id": _id, "text": q, "metadata": {}})
            train_qrels.append([_id, s, '1'])

with open(path_to_valid_questions) as fin:
    lines = fin.readlines()
    valid_qrels = []
    for i, l in enumerate(lines):
        s, p, o, q = lines[i].strip('\n').split('\t')
        _id = 'v' + str(i)
        if s in entities:
            queries.append({"_id": _id, "text": q, "metadata": {}})
            valid_qrels.append([_id, s, '1'])

print(len(queries), 'samples with entities recognised')
print(queries[0], train_qrels[0], valid_qrels[0])

26006 samples with entities recognised
{'_id': 't0', 'text': 'what movie is produced by warner bros.', 'metadata': {}} ['t0', 'Q126399', '1'] ['v1', 'Q318926', '1']


In [10]:
# save dataset
import jsonlines

with open(corpus_path, 'w') as out_file:
    for d in corpus:
        out_file.write(json.dumps(d))
        out_file.write("\n")
        
with open(query_path, 'w') as out_file:
    for d in queries:
        out_file.write(json.dumps(d))
        out_file.write("\n")

with open(train_qrels_path, 'w') as out_file:
    for qrel in train_qrels:
        out_file.write('\t'.join(qrel)+'\n')

with open(valid_qrels_path, 'w') as out_file:
    for qrel in valid_qrels:
        out_file.write('\t'.join(qrel)+'\n')

In [12]:
from beir.datasets.data_loader import GenericDataLoader

data_path = '/ivi/ilps/personal/svakule/spoken_qa/WD18_entities'
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="train")
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="valid")